# Fine tune a TF model

In [1]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

2023-03-11 02:52:51.164974: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 02:52:51.292683: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-11 02:52:51.292715: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-11 02:52:51.998495: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

## Prepare data

In [2]:
dataset = load_dataset("yelp_review_full")

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /home/jovyan/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [4]:
dataset['train'][10]

{'label': 0,
 'text': "Owning a driving range inside the city limits is like a license to print money.  I don't think I ask much out of a driving range.  Decent mats, clean balls and accessible hours.  Hell you need even less people now with the advent of the machine that doles out the balls.  This place has none of them.  It is april and there are no grass tees yet.  BTW they opened for the season this week although it has been golfing weather for a month.  The mats look like the carpet at my 107 year old aunt Irene's house.  Worn and thread bare.  Let's talk about the hours.  This place is equipped with lights yet they only sell buckets of balls until 730.  It is still light out.  Finally lets you have the pit to hit into.  When I arrived I wasn't sure if this was a driving range or an excavation site for a mastodon or a strip mining operation.  There is no grass on the range. Just mud.  Makes it a good tool to figure out how far you actually are hitting the ball.  Oh, they are cash 

In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [6]:
tokenizer('This is a ramdom text')

{'input_ids': [101, 1188, 1110, 170, 26084, 9277, 3087, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [7]:
def tokenize_sample(sample):
    return tokenizer(sample["text"], max_length=512, truncation=True)

In [8]:
tokenize_sample(dataset['train'][10])

{'input_ids': [101, 13432, 1158, 170, 3759, 2079, 1656, 1103, 1331, 6263, 1110, 1176, 170, 5941, 1106, 5911, 1948, 119, 146, 1274, 112, 189, 1341, 146, 2367, 1277, 1149, 1104, 170, 3759, 2079, 119, 13063, 3452, 22591, 1116, 117, 4044, 7318, 1105, 7385, 2005, 119, 5479, 1128, 1444, 1256, 1750, 1234, 1208, 1114, 1103, 16889, 1104, 1103, 3395, 1115, 1202, 2897, 1149, 1103, 7318, 119, 1188, 1282, 1144, 3839, 1104, 1172, 119, 1135, 1110, 170, 1643, 13217, 1105, 1175, 1132, 1185, 5282, 26425, 1116, 1870, 119, 27378, 2924, 1152, 1533, 1111, 1103, 1265, 1142, 1989, 1780, 1122, 1144, 1151, 7135, 1158, 4250, 1111, 170, 2370, 119, 1109, 22591, 1116, 1440, 1176, 1103, 10797, 1120, 1139, 10428, 1214, 1385, 8524, 13524, 112, 188, 1402, 119, 160, 8456, 1105, 12473, 6038, 119, 2421, 112, 188, 2037, 1164, 1103, 2005, 119, 1188, 1282, 1110, 5440, 1114, 4204, 1870, 1152, 1178, 4582, 15398, 1116, 1104, 7318, 1235, 5766, 1568, 119, 1135, 1110, 1253, 1609, 1149, 119, 4428, 11446, 1128, 1138, 1103, 7172, 110

In [9]:
encoded_dataset = dataset.map(tokenize_sample, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
pre_tokenizer_columns = set(dataset["train"].features)
tokenizer_columns = list(set(encoded_dataset["train"].features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

Columns added by tokenizer: ['token_type_ids', 'attention_mask', 'input_ids']


In [11]:
encoded_dataset["train"].features["label"]

ClassLabel(names=['1 star', '2 star', '3 stars', '4 stars', '5 stars'], id=None)

## Train model

In [12]:
from tensorflow.keras.optimizers import Adam
import numpy as np
from transformers.keras_callbacks import KerasMetricCallback
from tensorflow.keras.callbacks import TensorBoard

In [13]:
labels = encoded_dataset["train"].features["label"].names
num_epochs = 1

In [14]:
num_labels = len(labels)
num_labels

5

In [15]:
id2label = {v: k for v, k in enumerate(labels)}
label2id = {val: key for key, val in id2label.items()}

In [16]:
model = TFAutoModelForSequenceClassification.from_pretrained(
    'bert-base-cased', num_labels=num_labels, id2label=id2label, label2id=label2id
)

2023-03-11 02:55:05.874833: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-11 02:55:05.874871: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-11 02:55:05.874891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-9acf10): /proc/driver/nvidia/version does not exist
2023-03-11 02:55:05.875143: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 02:55:06.016020: W tensorflow/tsl/framework/cpu

In [17]:
tf_train_dataset = model.prepare_tf_dataset(
    encoded_dataset["train"],
    shuffle=True,
    batch_size=4,
    tokenizer=tokenizer
).take(100)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [18]:
tf_test_dataset = model.prepare_tf_dataset(
    encoded_dataset['test'],
    shuffle=False,
    batch_size=16,
    tokenizer=tokenizer,
).take(100)

In [19]:
model.compile()

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [20]:
def compute_metrics(eval_predictions):
    predictions, labels = eval_predictions
    return metric.compute(predictions=predictions, references=labels)

In [21]:
metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=tf_test_dataset
)

In [22]:
model_name = "fined_tuned_bert_based"

In [23]:
tensorboard_callback = TensorBoard(log_dir="./text_classification_model_save/logs")

In [24]:
callbacks = [metric_callback, tensorboard_callback]

In [25]:
model.fit(
    tf_train_dataset,
    validation_data=None,
    epochs=num_epochs,
    callbacks=callbacks,
)

100/100 [==============================] - ETA: 0s - loss: 1.8808

NameError: name 'metric' is not defined

In [26]:
model

In [41]:
sentenses = ["I got \'new\' tires from them and within two weeks got a flat", "Food was NOT GOOD at all! My husband & I ate here a couple weeks ago for the first time."]

In [42]:
inputs = tokenizer(sentenses, return_tensors="np", padding="longest")

In [43]:
model(inputs).logits

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[-0.00887552, -0.88318735, -0.51978284,  0.0714431 , -1.141663  ],
       [-0.00887532, -0.883187  , -0.5197825 ,  0.07144319, -1.1416627 ]],
      dtype=float32)>